### Deep learning inference

- Detect tree objects from dataset

In [ ]:
from urbantree.setting import Setting

#SETTING = "settings/by_dop80c_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r2/setting.yaml"
SETTING = "settings/by_dop80c_1312/deepforest_r3/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml"

setting = Setting.load_deepforest_setting(SETTING)
setting['model_inference_config']

- detect tree objects from dataset and the resulting bounding boxes DataFrame are saved in pickle objects.
- The object detection can be performed on different patch sizes and
  no extra nms is run on aggregated results.

In [ ]:
from urbantree.deepforest.detection import infer_images

infer_images(**setting)

- Render tree canopy raster images according to the resulting bounding boxes DataFrame saved in pickle objects.

In [ ]:
from urbantree.deepforest.detection import postprocess_render_images

dataset_img_pattern="*.tiff"
#dataset_img_pattern = "*6128569*"
#dataset_img_pattern = "*1287912.570476842_*"

postprocess_render_images(**setting,
                          dataset_img_pattern=dataset_img_pattern)

- calculate difference among two datasets

In [1]:
from urbantree.deepforest.detection import calc_diff

# tune inference parameter
DIFF_FROM_INFERENCE_PARAM = {
  'confident_min_bbox_size': 64, 
  'confident_min_score': 0.6, 
  'morphology_factor': 1, 
  'concurrency': 4, 
  'patch': [
    {
      'patch_size': 1200, 
      'patch_overlap': 0.3, 
      'iou_threshold': 0.8, 
      'score_thresh': 0.9
    }, 
    {
      'patch_size': 800, 
      'patch_overlap': 0.3, 
      'iou_threshold': 0.8, 
      'score_thresh': 0.48
    }, 
    {
      'patch_size': 200, 
      'patch_overlap': 0.2, 
      'iou_threshold': 0.7, 
      'score_thresh': 0.455
    }, 
    {
      'patch_size': 96, 
      'patch_overlap': 0.18, 
      'iou_threshold': 0.6, 
      'score_thresh': 0.525
    }
  ]
}
DIFF_TO_INFERENCE_PARAM = {
  'confident_min_bbox_size': 64,
  'confident_min_score': 0.6, 
  'morphology_factor': 1, 
  'concurrency': 4, 
  'patch': [
    {
      'patch_size': 1200, 
      'patch_overlap': 0.3, 
      'iou_threshold': 0.8, 
      'score_thresh': 0.6
    }, 
    {
      'patch_size': 800, 
      'patch_overlap': 0.3, 
      'iou_threshold': 0.8, 
      'score_thresh': 0.36
    }, 
    {
      'patch_size': 200, 
      'patch_overlap': 0.2, 
      'iou_threshold': 0.7, 
      'score_thresh': 0.24
    }, 
    {
      'patch_size': 96, 
      'patch_overlap': 0.18, 
      'iou_threshold': 0.6, 
      'score_thresh': 0.21
    }
  ]
}

calc_diff(diff_from_setting_path="settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml",
          diff_to_setting_path="settings/by_dop80c_1312/deepforest_r3/setting.yaml",
          diff_from_inference_param=DIFF_FROM_INFERENCE_PARAM,
          diff_to_inference_param=DIFF_TO_INFERENCE_PARAM,
          aggregate_iou_threshold=0.4,
          diff_iou_threshold=0.1,
          diff_cover_threshold=0.4,
          output_bbox_dir='temp/diff4/b',
          output_debug_img_dir='temp/diff4/debug',
          concurrency=6) 

DIFF_FROM_INFERENCE_PARAM {'confident_min_bbox_size': 64, 'confident_min_score': 0.6, 'morphology_factor': 1, 'concurrency': 4, 'patch': [{'patch_size': 1200, 'patch_overlap': 0.3, 'iou_threshold': 0.8, 'score_thresh': 0.9}, {'patch_size': 800, 'patch_overlap': 0.3, 'iou_threshold': 0.8, 'score_thresh': 0.48}, {'patch_size': 200, 'patch_overlap': 0.2, 'iou_threshold': 0.7, 'score_thresh': 0.455}, {'patch_size': 96, 'patch_overlap': 0.18, 'iou_threshold': 0.6, 'score_thresh': 0.525}]}
DIFF_TO_INFERENCE_PARAM {'confident_min_bbox_size': 64, 'confident_min_score': 0.6, 'morphology_factor': 1, 'concurrency': 4, 'patch': [{'patch_size': 1200, 'patch_overlap': 0.3, 'iou_threshold': 0.8, 'score_thresh': 0.6}, {'patch_size': 800, 'patch_overlap': 0.3, 'iou_threshold': 0.8, 'score_thresh': 0.36}, {'patch_size': 200, 'patch_overlap': 0.2, 'iou_threshold': 0.7, 'score_thresh': 0.24}, {'patch_size': 96, 'patch_overlap': 0.18, 'iou_threshold': 0.6, 'score_thresh': 0.21}]}


In [ ]:
from urbantree.deepforest.detection import create_bbox_geojson

create_bbox_geojson(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_dif='temp/diff4/b/diff',
                    output_geojson_path='temp/diff4/diff.geojson',
                    iou_threshold=0.3)
create_bbox_geojson(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_dif='temp/diff4/b/from',
                    output_geojson_path='temp/diff4/from.geojson',
                    iou_threshold=0.3)
create_bbox_geojson(src_img_dir='aerial_images_resampled/by_dop80c_1312',
                    src_bbox_dif='temp/diff4/b/to',
                    output_geojson_path='temp/diff4/to.geojson',
                    iou_threshold=0.3)
